**AI Usage**

*Tool:* Chat GPT (GPT-5)

*Purpose:* Debugged errors; explained XGboost

*Usage:* Adopted ideas on how to implement XGboost; modified code for XGboost inputs and coverage plot mapie regressor; explained how to split date time column; debugged error related to getting dummies for hour column

*Location:* Documented here and further comments in traffic.py

## **Import Libraries**


In [18]:
import pandas as pd                  # Pandas
import numpy as np                   # Numpy
from matplotlib import pyplot as plt # Matplotlib
import seaborn as sns                # Seaborn

# Package to implement ML Algorithms
import sklearn
from xgboost import XGBRegressor    # XGBoost Regressor

# Package for data partitioning
from sklearn.model_selection import train_test_split

# Import MAPIE to calculate prediction intervals
from mapie.regression import MapieRegressor

# To calculate coverage score
from mapie.metrics import regression_coverage_score

# Module to save and load Python objects to and from files
import pickle 

# Import necessary library for setting up the plot format
import matplotlib as mpl

# Ignore Deprecation Warnings
import warnings
warnings.filterwarnings('ignore')

# Display inline plots as vector-based (svg)
%config InlineBackend.figure_formats = ['svg']

%matplotlib inline

In [19]:
traffic_df = pd.read_csv('Traffic_Volume.csv')
traffic_df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume
0,NaN,288.28,0.0,0.0,40,Clouds,10/2/12 9:00,5545
1,NaN,289.36,0.0,0.0,75,Clouds,10/2/12 10:00,4516
2,NaN,289.58,0.0,0.0,90,Clouds,10/2/12 11:00,4767
3,NaN,290.13,0.0,0.0,90,Clouds,10/2/12 12:00,5026
4,NaN,291.14,0.0,0.0,75,Clouds,10/2/12 13:00,4918


In [20]:
# breaking down the date time column into multiple columns
traffic_df['date_time'] = pd.to_datetime(traffic_df['date_time'], format='%m/%d/%y %H:%M')
traffic_df['hour'] = traffic_df['date_time'].dt.hour
traffic_df['weekday'] = traffic_df['date_time'].dt.day_name()
traffic_df['month'] = traffic_df['date_time'].dt.month_name()

traffic_df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,date_time,traffic_volume,hour,weekday,month
0,NaN,288.28,0.0,0.0,40,Clouds,2012-10-02 09:00:00,5545,9,Tuesday,October
1,NaN,289.36,0.0,0.0,75,Clouds,2012-10-02 10:00:00,4516,10,Tuesday,October
2,NaN,289.58,0.0,0.0,90,Clouds,2012-10-02 11:00:00,4767,11,Tuesday,October
3,NaN,290.13,0.0,0.0,90,Clouds,2012-10-02 12:00:00,5026,12,Tuesday,October
4,NaN,291.14,0.0,0.0,75,Clouds,2012-10-02 13:00:00,4918,13,Tuesday,October


**Selecting Input and Output Features**

In [21]:
# Select input and output features
features = traffic_df.drop(columns = ['traffic_volume', 'date_time'])
output = traffic_df['traffic_volume']

In [22]:
# One hot encoding for categorical variables
cat = ['holiday', 'weather_main', 'weekday', 'month', 'hour']
features_encoded = pd.get_dummies(features, columns=cat, drop_first=True)
features_encoded.head()



,temp,rain_1h,snow_1h,clouds_all,holiday_Columbus Day,holiday_Independence Day,holiday_Labor Day,holiday_Martin Luther King Jr Day,holiday_Memorial Day,holiday_New Years Day,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,288.28,0.0,0.0,40,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,289.36,0.0,0.0,75,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,289.58,0.0,0.0,90,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,290.13,0.0,0.0,90,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,291.14,0.0,0.0,75,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


**Data Partitioning**

In [23]:
train_X, test_X, train_y, test_y = train_test_split(features_encoded, output, test_size = 0.2, random_state = 1) 


## **Predicting using XGBoost**

In [24]:
# Defining prediction model
reg_xg = XGBRegressor(random_state=42)

# Fitting model on training data
reg_xg.fit(train_X, train_y);

In [25]:
# Predict Test Set
y_pred = reg_xg.predict(test_X)

# evaluate the model on test set
r2 = sklearn.metrics.r2_score(test_y, y_pred)
print('R-squared on Test Set: %0.2f' %r2)

RMSE_test = sklearn.metrics.root_mean_squared_error(test_y, y_pred)
print('RMSE on Test Set: %0.2f' %RMSE_test)

y_train_pred = reg_xg.predict(train_X)
# evaluate the model on test set
r2 = sklearn.metrics.r2_score(train_y, y_train_pred)
print('R-squared on Test Set: %0.2f' %r2)

RMSE_test = sklearn.metrics.root_mean_squared_error(train_y, y_train_pred)
print('RMSE on Test Set: %0.2f' %RMSE_test)

R-squared on Test Set: 0.93
RMSE on Test Set: 537.51
R-squared on Test Set: 0.94
RMSE on Test Set: 497.28


### **Model Insights**

**Feature Importance**

In [26]:
# Storing importance values from the trained model
importance = reg_xg.feature_importances_

# Storing feature importance as a dataframe
feature_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
               columns = ['Feature', 'Importance'])

feature_imp = feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
plt.figure(figsize = (12, 8))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color = ['blue', 'yellow'])

plt.xlabel("Importance")
plt.ylabel("Input Feature")
plt.title('Which features are the most important for species prediction?') 
plt.tight_layout()
plt.savefig("xg_feature_imp.svg");

**Histogram of Residuals**

In [27]:
# Calculate the residuals by subtracting the predicted values from the actual test values
all_residuals = test_y - y_pred

# Set up the figure with custom size and resolution (DPI)
plt.figure(figsize=(6, 4), dpi = 150)

# Plot the histogram of residuals
plt.hist(all_residuals, bins = 25, color = 'lime', edgecolor = 'black')

# Label X and Y axes
plt.xlabel('Residuals', fontsize = 14)
plt.ylabel('# of Test Datapoints', fontsize = 14)

# Set the title of the plot
plt.title('Distribution of Residuals', fontsize = 16)

# Adjust the font size of x and y ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.savefig("hist_residuals.svg");

**Predicted vs Actual**

In [28]:
# Setting the figure size and resolution
plt.figure(figsize = (6, 4), dpi = 150)

# Scatter plot of actual vs predicted values
plt.scatter(test_y, y_pred, color = 'purple', alpha = 0.6, edgecolor = 'black', s = 40)

# 45-degree reference line (perfect predictions)
plt.plot([min(test_y), max(test_y)], [min(test_y), max(test_y)], color = 'red', linestyle = '--', lw = 2)

# Axis labels and title
plt.xlabel('Actual Values', fontsize = 10)
plt.ylabel('Predicted Values', fontsize = 10)
plt.title('Predicted vs Actual Values', fontsize = 12)

# Adjust ticks
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)

plt.savefig("pred_actual.svg");

**Coverage Plot**

In [29]:
# Define mapie regressor
mapie = MapieRegressor(
    estimator=reg_xg,
    method="minmax",
    cv=5,
    random_state=42
)


In [30]:
# Use a calibration subset
subset = train_X.sample(10000, random_state=42)
mapie.fit(subset, train_y.loc[subset.index])

MapieRegressor(cv=5,
               estimator=XGBRegressor(base_score=None, booster=None,
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=False,
                                      eval_metric=None, feature_types=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=None, max_bin=None,
                                      max_cat_threshold=None,
                                      max_cat_to_onehot=None,
                                      max_delta_step=None, max_depth=None,
                                      max_leaves=None, min_child_weight=None,
                                      missing=nan, monotone_constraints=None,
                                      multi_strategy=None, n_estimators=None,
                                      n_jobs=None, num_parallel_tree=None,
                                      random_state=42, ...),
               method='minmax', random_state=42)

In [31]:
alpha = 0.1 # For 90% confidence level

# Use mapie.predict() to get predicted values and intervals
y_test_pred, y_test_pis = mapie.predict(test_X, alpha = alpha)

# Storing results in a dataframe
predictions = test_y.to_frame()
predictions.columns = ['Actual Value']
predictions["Predicted Value"] = y_test_pred.round()
predictions["Lower Value"] = y_test_pis[:, 0].round()
predictions["Upper Value"] = y_test_pis[:, 1].round()

In [32]:
coverage = regression_coverage_score(test_y,           # Actual values
                                     y_test_pis[:, 0], # Lower bound of prediction intervals
                                     y_test_pis[:, 1]) # Upper bound of prediction intervals

coverage_percentage = coverage * 100
print(f"Coverage: {coverage_percentage:.2f}%")

Coverage: 93.43%


In [33]:
# Sort the predictions by 'Actual Value' for better visualization and reset the index
sorted_predictions = predictions.sort_values(by=['Actual Value']).reset_index(drop=True)

# Create a figure and axis object with specified size and resolution
fig, ax = plt.subplots(figsize=(6, 4), dpi=150)

# Plot the actual values with green dots
plt.plot(sorted_predictions["Actual Value"], 'bo', markersize=4, label="Actual Value")

# Fill the area between the lower and upper bounds of the prediction intervals with semi-transparent green color
plt.fill_between(np.arange(len(sorted_predictions)),
                 sorted_predictions["Lower Value"],
                 sorted_predictions["Upper Value"],
                 alpha=0.2, color="blue", label="Prediction Interval")

# Set font size for x and y ticks
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Format y-axis to show values with commas as thousand separators
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

# Set the limit for the x-axis to cover the range of samples
plt.xlim([0, len(sorted_predictions)])

# Label the x-axis and y-axis with appropriate font size
plt.xlabel("Samples", fontsize=10)
plt.ylabel("Target", fontsize=10)

# Add a title to the plot, including the coverage percentage, with bold formatting
plt.title(f"Prediction Intervals and Coverage: {coverage_percentage:.2f}%", fontsize=12, fontweight="bold")

# Add a legend to the plot, placed in the upper left, with specified font size
plt.legend(loc="upper left", fontsize=10)

# Save the plot as a PDF file with tight layout
plt.savefig("coverage.svg", format="svg", bbox_inches="tight");


**Save Pickle**

In [34]:
# Pickle file: saving the trained DT model
# Creating the file where we want to write the model (wb = write binary)
xg_pickle = open('traffic.pickle', 'wb') 

# Write DT model to the file
pickle.dump(mapie, xg_pickle) 

# Close the file
xg_pickle.close() 